In [ ]:
from kaggle_secrets import UserSecretsClient
import os
secret_names = ["OPENAI_API_KEY","LANGCHAIN_TRACING_V2", "TAVILY_API_KEY", "HUGGINGFACEHUB_API_TOKEN"]
user_secrets = UserSecretsClient()
for secret_name in secret_names:
    # Retrieve the secret value
    secret_value = user_secrets.get_secret(secret_name)
    # Set the secret as an environment variable
    os.environ[secret_name] = secret_value

In [ ]:
!pip install langchain

In [ ]:
!pip install openai

In [ ]:
# Prompts and LLMs
from langchain.llms import OpenAI

# Temparature lies between 0 and 1
llm = OpenAI(temperature=0.9)
prompt = "what would a good company name be for a company that makes colorful socks?"

In [ ]:
print(llm(prompt))

In [ ]:
result = llm.generate([prompt]*5)
for company_name in result.generations:
    print(company_name[0].text)

In [ ]:
result.generations

In [ ]:
# using different Models with Langchain
!pip install huggingface_hub

In [ ]:
from langchain import HuggingFaceHub

llm = HuggingFaceHub(repo_id="google/flan-t5-base", model_kwargs={"temperature": 0.9, "max_length": 64})

prompt = "What are good fitness tips?"

In [ ]:
print(llm(prompt))

In [ ]:
# Prompt Templating and chaining 
from langchain import PromptTemplate
from langchain.llms import OpenAI
from langchain.chains import LLMChain

template = "You are a naming consultant for new companies. What is good name for a {company} that makes {product}?"

In [ ]:
prompt = PromptTemplate.from_template(template)
print(prompt.format(product="colorful socks"))

In [ ]:
llm = OpenAI(temperature=0.9)
chain = LLMChain(llm=llm , prompt=prompt)

In [ ]:
print(chain.run({'company':"ABC",'product':"colorful socks"}))

In [ ]:
# Simple Sequential Chains 
from langchain import OpenAI, LLMChain, PromptTemplate
from langchain.chains import SimpleSequentialChain

llm = OpenAI(temperature=0)
template = "What is a good name for a company that makes {product}?"
first_prompt = PromptTemplate.from_template(template)
first_chain = LLMChain(llm=llm, prompt=first_prompt)

In [ ]:
print(first_chain.run("colorfulsocks"))

In [ ]:
second_template = "Write a catch phrase for the following company: {company_name}"
second_prompt = PromptTemplate.from_template(second_template)
second_chain = LLMChain(llm=llm, prompt=second_prompt)

In [ ]:
print(second_chain.run(first_chain.run("colorful socks")))

In [ ]:
overall_chain = SimpleSequentialChain(chains=[first_chain, second_chain], verbose=True)

In [ ]:
catchphrase = overall_chain.run("colorful socks")

In [ ]:
print(catchphrase)

In [ ]:
# Action Agents

prompt = "When was the 3rd president of the united states born? what is that year raised to the power of 3?"

In [ ]:
pip install wikipedia

In [ ]:
import pprint
from langchain.agents import get_all_tool_names
pp = pprint.PrettyPrinter(indent=4)
pp.pprint(get_all_tool_names())

In [ ]:
from langchain.llms import OpenAI
from langchain.agents import load_tools, initialize_agent

llm = OpenAI(temperature=0)
tools = load_tools(["wikipedia","llm-math"], llm=llm)
agent = initialize_agent(tools, llm, agent="zero-shot-react-description", verbose=True)

agent.run(prompt)

In [ ]:
from langchain.agents import AgentType

tools = load_tools(["human"])
agent_chain = initialize_agent(
tools,
llm,
agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
verbose=True
)

agent_chain.run("What's my friend Andi's surname")

In [ ]:
# Plan and Execute Agent 
# First plan the series of steps, and then execute each one 

prompt = "Where are the next summer olympics going to be hosted? What is the population of that country raised to the 0.43 power?"

# Search for the location
# Identify the country
# Search for the population of the country 
# Raise the population to .43
# Return the result


In [ ]:
from langchain_community.tools.tavily_search import TavilySearchResults

search = TavilySearchResults()

In [ ]:
!pip install langchain_experimental

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain_experimental.plan_and_execute import PlanAndExecute, load_agent_executor, load_chat_planner
from langchain import LLMMathChain, WikipediaAPIWrapper
from langchain.agents.tools import Tool

llm_math_chain = LLMMathChain.from_llm(llm=llm,verbose=True)
wikipedia = WikipediaAPIWrapper()

tools = [
    Tool(
    name="Search",
    func=search.run,
    description="useful for when you need to answer questions about current events"
    ),
    Tool(
    name="Wikipedia",
    func=wikipedia.run,
    description="useful for when you need to look up facts and statistics"
    ),
    Tool(
    name="Calculator",
    func=llm_math_chain.run,
    description="useful for when you need to answer questions about math"
    ),
]

# Planner and executor should use memory i.e. chat models
model = ChatOpenAI(temperature=0)
planner = load_chat_planner(model)
executor = load_agent_executor(model, tools, verbose=True)
agent = PlanAndExecute(planner=planner, executor= executor, verbose=True)
agent.run(prompt)


In [ ]:
# Memory and Chat Bots

from langchain import OpenAI,ConversationChain

llm = OpenAI(temperature=0)
conversation = ConversationChain(llm=llm, verbose=True)

print(conversation.predict(input="Hi there!"))

In [ ]:
print(conversation.predict(input="Can we talk about the weather?"))

In [ ]:
print(conversation.predict(input="It's a beautiful day"))

In [ ]:
# Memory and Chat Bots

from langchain import OpenAI,ConversationChain

llm = OpenAI(temperature=0)
conversation = ConversationChain(llm=llm, verbose=True)

print("Welcome to you AI Chatbot! What's on your mind?")
for _ in range(0,3):
    human_input = input("you: ")
    ai_response = conversation.predict(input=human_input)
    print(f"AI: {ai_response}")

In [ ]:
from langchain.memory import ChatMessageHistory, ConversationBufferMemory
from langchain.schema import  messages_from_dict, messages_to_dict

history = ChatMessageHistory()
history.add_user_message("hello! let's talk about giraffes")
history.add_ai_message("hi! I'm down to talk about giraffes")
dicts = messages_to_dict(history.messages)
print(dicts)

In [ ]:
new_messages = messages_from_dict(dicts)
llm = OpenAI(temperature=0)
history = ChatMessageHistory(messages=new_messages)
buffer = ConversationBufferMemory(chat_memory=history)
conversation = ConversationChain(llm=llm, memory = buffer, verbose=True)
print(conversation.predict(input="What are they?"))

In [ ]:
print(conversation.memory)

In [ ]:
# Document Loading And QA Retrieval
!pip install chromadb

In [ ]:
!pip install tiktoken

In [ ]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA


In [ ]:
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://python.langchain.com/v0.1/docs/get_started/quickstart/")

documents = loader.load()

In [ ]:
print(documents)

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
texts = text_splitter.split_documents(documents)

In [ ]:
print(texts[0])

In [ ]:
embeddings = OpenAIEmbeddings()
store = Chroma.from_documents(texts, embeddings, collection_name="quick-start")

In [ ]:
llm = OpenAI(temperature=0)
chain = RetrievalQA.from_chain_type(llm, retriever=store.as_retriever())
print(chain.run("What is Conversation Retrieval Chain in langchain?"))